# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()
spark

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [3]:
path = 'Datasets/'

playStore = spark.read.csv(path + 'googleplaystore.csv',inferSchema=True,header=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [4]:
playStore.limit(5).toPandas()

c:\users\aditya\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


App        Category Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN    4.1   
1                                Coloring book moana  ART_AND_DESIGN    3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN    4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN    4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN    4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend Play  January 15, 2018               2.0.0   
2               Art & Design    August 1, 2018               1.2.4   
3               Art & Design      June 8, 2018  Varies with device   
4    Art & Design;Creativity     June 20, 2018                 1.1   

    Android Ver  
0  4.0.3 and up  
1  4.0.3 and up  
2  4.0.3 and up  
3    4.2 and up  
4    4.4 and up

As well as the schema to make sure all the column types were correctly infered

In [5]:
playStore.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [6]:
from pyspark.sql.types import IntegerType, FloatType
newdf = playStore.withColumn("Rating", playStore["Rating"].cast(FloatType())) \
            .withColumn("Reviews", playStore["Reviews"].cast(IntegerType())) \
            .withColumn("Price", playStore["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

   Reviews  Size     Installs  Type  Price Content Rating  \
0      159   19M      10,000+  Free      0       Everyone   
1      967   14M     500,000+  Free      0       Everyone   
2    87510  8.7M   5,000,000+  Free      0       Everyone   
3   215644   25M  50,000,000+  Free      0           Teen   
4      967  2.8M     100,000+  Free      0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend Play  January 15, 2018               2.0.0   
2               Art & Design    August 1, 2018               1.2.4   
3               Art & Design      June 8, 2018  Varies with device   
4    Art & Design;Creativity     June 20, 2018                 1.1   

    Android Ver  
0  4.0.3 and up  
1  4.0.3 and up  
2  4.0.3 and up  
3    4.2 and up  
4    4.4 and up

Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [7]:
newdf.createOrReplaceTempView('tempView')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [17]:
sqlResult = spark.sql('Select * from tempView where Rating> 4.1')
sqlResult.limit(5).toPandas()

App        Category  Rating  \
0  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
1                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
2              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   
3                         Paper flowers instructions  ART_AND_DESIGN     4.4   
4                               Garden Coloring Book  ART_AND_DESIGN     4.4   

   Reviews  Size     Installs  Type  Price Content Rating  \
0    87510  8.7M   5,000,000+  Free      0       Everyone   
1   215644   25M  50,000,000+  Free      0           Teen   
2      967  2.8M     100,000+  Free      0       Everyone   
3      167  5.6M      50,000+  Free      0       Everyone   
4    13791   33M   1,000,000+  Free      0       Everyone   

                    Genres        Last Updated         Current Ver  \
0             Art & Design      August 1, 2018               1.2.4   
1             Art & Design        June 8, 2018  Varies with device   
2  Art & Design;Creativity       June 20, 2018                 1.1   
3             Art & Design      March 26, 2017                 1.0   
4             Art & Design  September 20, 2017               2.9.2   

    Android Ver  
0  4.0.3 and up  
1    4.2 and up  
2    4.4 and up  
3    2.3 and up  
4    3.0 and up

## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [26]:
sqlResult2 = spark.sql("Select App, Rating from tempView where Category ='COMICS' and Rating> 4.5")
sqlResult2.limit(5).toPandas()

App  Rating
0           Manga Master - Best manga & comic reader     4.6
1  GANMA! - All original stories free of charge f...     4.7
2                          Röhrich Werner Soundboard     4.7
3                    Unicorn Pokez - Color By Number     4.8
4                      Manga - read Thai translation     4.6

## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [32]:
sqlResult3 = spark.sql("Select Category, sum(Reviews) as Total_views from tempView group by Category order by Total_views desc")
sqlResult3.limit(1).toPandas()

Category  Total_views
0     GAME   1585422349

## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [34]:
sqlResult4 = spark.sql("Select App, Reviews from tempView order by Reviews desc")
sqlResult4.limit(1).toPandas()

App   Reviews
0  Facebook  78158306

## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [38]:
sqlResult5 = spark.sql("Select App from tempView where App like '%dating%' ")
sqlResult5.toPandas()

App
0  Meet, chat & date. Free dating app - Chocolate...
1          Friend Find: free chat + flirt dating app
2                              Spine- The dating app
3      Princess Closet : Otome games free dating sim
4                           happn – Local dating app

## 6. Use SQL Transformer to display how many free apps there are in this list

In [43]:
from pyspark.ml.feature import SQLTransformer

sqlTransform = SQLTransformer(statement = "select count(*) from tempView where Type = 'Free'")

In [45]:
sqlTransform.transform(newdf).show()

+--------+
|count(1)|
+--------+
|   10037|
+--------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [55]:
sqlTransform2 = SQLTransformer(statement = "select Genres, count(*) as Total from __THIS__ group by Genres order by Total desc")

In [56]:
sqlTransform2.transform(newdf).show(1)

+------+-----+
|Genres|Total|
+------+-----+
| Tools|  842|
+------+-----+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [61]:
sqlTransform3 = SQLTransformer(statement = "select App, Reviews from __THIS__ where Genres= 'Tools' and Reviews > 100 ")

In [62]:
sqlTransform3.transform(newdf).show(5)

+-----------------+-------+
|              App|Reviews|
+-----------------+-------+
|Moto File Manager|  38655|
|           Google|8033493|
| Google Translate|5745093|
|     Moto Display|  18239|
|   Motorola Alert|  24199|
+-----------------+-------+
only showing top 5 rows



## That's all folks! Great job!